In [2]:
import numpy as np 

u = np.arange(30)
L = 30
# print(u)
# print(Hankel_matrix(u,L))

In [3]:
import redis
import pickle
import numpy as np
import pandas as pd
from util import Matrix_inv_phase1
import os

rs = redis.StrictRedis(host='127.0.0.1',db=2, port=6379,password="chlpw1039")
cav_id = 0
keys = [
    f'Uip_in_CAV_{cav_id}',
    f'Uif_in_CAV_{cav_id}',
    f'Eip_in_CAV_{cav_id}',
    f'Eif_in_CAV_{cav_id}',
    f'Yip_in_CAV_{cav_id}',
    f'Yif_in_CAV_{cav_id}',
    # f'Uip_1_in_CAV_{cav_id}',
    # f'Uif_1_in_CAV_{cav_id}',
    # f'Eip_1_in_CAV_{cav_id}',
    # f'Eif_1_in_CAV_{cav_id}',
    # f'Yip_1_in_CAV_{cav_id}',
    # f'Yif_1_in_CAV_{cav_id}',
    f'lambda_yi',
    f'lambda_gi',
    f'rho',
    f'g_initial_in_CAV_{cav_id}',
    f'mu_initial_in_CAV_{cav_id}',
    f'eta_initial_in_CAV_{cav_id}',
    f'phi_initial_in_CAV_{cav_id}',
    f'theta_initial_in_CAV_{cav_id}',
    f'Su_in_CAV_{cav_id}',
    f'Sy_in_CAV_{cav_id}',
    f'Se_in_CAV_{cav_id}',
    f'n_cav',
    f'k_on',
    f'Qi_in_CAV_{cav_id}',
    f'Ri_in_CAV_{cav_id}',
    f'K_in_CAV_{cav_id}',
    f'P_in_CAV_{cav_id}',
    f'M_in_CAV_{cav_id}',
    f'acel_max',
    f'dcel_max',
    f's_max',
    f's_st',
    f's_star',
    f'Tstep',
]

values = rs.mget(keys)

(
    # Uip, Uif, Eip, Eif, Yip, Yif,
    Uip1, Uif1, Eip1, Eif1, Yip1, Yif1,
    lambda_yi, lambda_gi, rho,
    g_initial, mu_initial, eta_initial, phi_initial, theta_initial, 
    Su, Sy, Se, 
    n_cav, k_on, Qi_stack, Ri_stack, K, P, M, 
    acel_max, dcel_max, spacing_max, spacing_min, s_star, Tstep
) = [pickle.loads(value) for value in values]
kappa = int(Uip1.shape[1])

# variable_names = [
#     'Uip', 'Uif', 'Eip', 'Eif', 'Yip', 'Yif',
#     'Uip1', 'Uif1', 'Eip1', 'Eif1', 'Yip1', 'Yif1',
#     'lambda_yi', 'lambda_gi', 'rho', 'g_initial', 'mu_initial',
#     'eta_initial', 'phi_initial', 'theta_initial',
#     'Su', 'Sy', 'Se', 'n_cav', 'k_on',
#     'Qi_stack', 'Ri_stack', 'K', 'P', 'M',
#     'acel_max', 'dcel_max', 'spacing_max', 'spacing_min', 's_star', 'Tstep'
# ]

# # 获取值
# values = rs.mget(keys)
# variables = [pickle.loads(value) for value in values]

# # 创建文件夹存储 CSV 文件
# output_dir = "csv_variables"
# os.makedirs(output_dir, exist_ok=True)

# # 保存每个变量为单独的 CSV 文件
# for var_name, var in zip(variable_names, variables):
#     file_path = os.path.join(output_dir, f"{var_name}.csv")  # 使用变量名作为文件名
#     if isinstance(var, np.ndarray):
#         np.savetxt(file_path, var, delimiter=",")  # 保存多维数组
#     else:
#         with open(file_path, "w") as f:
#             f.write(str(var))  # 保存标量
# print(f"所有变量已保存到 {output_dir} 文件夹中")

# 直接求逆
Hgi_1 = Yif1.T@Qi_stack@Yif1+Uif1.T@Ri_stack@Uif1+\
        lambda_gi*np.eye(kappa)+lambda_yi*Yip1.T@Yip1+\
        rho/2*(np.eye(int(kappa))+Yif1.T@P.T@P@Yif1+Uif1.T@Uif1)+\
            rho/2*(Eif1.T@Eif1+Uip1.T@Uip1+Eip1.T@Eip1)
KKT_vert_1 = np.linalg.pinv(Hgi_1)
print('a=',KKT_vert_1)

#递推求逆
KKT_vert = pickle.loads(rs.mget(f'Hgi_vert_2_in_CAV_{cav_id}')[0])
print('b=',KKT_vert)

error = KKT_vert_1-KKT_vert
error_max = np.max(np.abs(error))
print(error_max)

a= [[ 0.10238463 -0.00164664  0.0037712  ...  0.00170342  0.00203448
   0.00194788]
 [-0.00164664  0.10201506 -0.00129528 ...  0.00131655  0.00223147
   0.00259124]
 [ 0.0037712  -0.00129529  0.10106976 ...  0.00114145  0.00202759
   0.00299654]
 ...
 [ 0.00170342  0.00131655  0.00114145 ...  0.12909087 -0.02483968
  -0.02578868]
 [ 0.00203448  0.00223147  0.00202759 ... -0.02483968  0.12253679
  -0.03277755]
 [ 0.00194788  0.00259124  0.00299655 ... -0.02578867 -0.03277755
   0.11280561]]
b= [[ 9.76919581e-02 -5.99147357e-03 -9.28815537e-04 ... -4.40100832e-04
   1.40667168e-04 -1.27376030e-04]
 [-5.97143414e-03  9.78261123e-02 -5.82969001e-03 ... -2.59848078e-04
   7.38903210e-05 -6.59335604e-05]
 [-9.30780495e-04 -5.85411295e-03  9.59834410e-02 ... -4.59866086e-04
   2.60538408e-04 -2.37359341e-04]
 ...
 [-5.89776465e-04 -3.07753870e-04 -5.96397410e-04 ... -1.66326605e-02
  -1.67679365e-03  1.16552482e-03]
 [ 1.83531001e-04  7.98714405e-05  2.94336421e-04 ... -1.55869741e-03
  -1.19

In [74]:
import redis
import pickle
import numpy as np

lambda_gi_on = np.linspace(100, 60, 431-81)
off_col = 20
lambda_gi_max = 100
kappa = 431

print(np.concatenate((np.full(off_col, lambda_gi_max), lambda_gi_on[:kappa-off_col])))


[100.          99.88538682  99.77077364  99.65616046  99.54154728
  99.4269341   99.31232092  99.19770774  99.08309456  98.96848138
  98.85386819  98.73925501  98.62464183  98.51002865  98.39541547
  98.28080229  98.16618911  98.05157593  97.93696275  97.82234957
  97.70773639  97.59312321  97.47851003  97.36389685  97.24928367
  97.13467049  97.02005731  96.90544413  96.79083095  96.67621777
  96.56160458  96.4469914   96.33237822  96.21776504  96.10315186
  95.98853868  95.8739255   95.75931232  95.64469914  95.53008596
  95.41547278  95.3008596   95.18624642  95.07163324  94.95702006
  94.84240688  94.7277937   94.61318052  94.49856734  94.38395415
  94.26934097  94.15472779  94.04011461  93.92550143  93.81088825
  93.69627507  93.58166189  93.46704871  93.35243553  93.23782235
  93.12320917  93.00859599  92.89398281  92.77936963  92.66475645
  92.55014327  92.43553009  92.32091691  92.20630372  92.09169054
  91.97707736  91.86246418  91.747851    91.63323782  91.51862464
  91.40401